In [1]:
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

print("✅ Configuration matplotlib activée")

✅ Configuration matplotlib activée


In [2]:
%matplotlib inline
# Installation des bibliothèques 
!pip install google-cloud-bigquery pandas matplotlib seaborn --quiet

In [3]:
# Imports
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration pour de beaux graphiques
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Bibliothèques importées avec succès !")

✅ Bibliothèques importées avec succès !


In [4]:
# Connexion à BigQuery
project_id = 'students-group1'
client = bigquery.Client(project=project_id)

print(f"✅ Connecté au projet : {project_id}")

✅ Connecté au projet : students-group1


In [5]:
# Charger la table movies
query_movies = """
SELECT *
FROM `master-ai-cloud.MoviePlatform.movies`
"""

df_movies = client.query(query_movies).to_dataframe()
print(f"✅ Table movies chargée : {len(df_movies)} films")
print(df_movies.head())

✅ Table movies chargée : 10329 films
   movieId                             title              genres
0   126929              Li'l Quinquin (    )  (no genres listed)
1   135460                      Pablo (2012)  (no genres listed)
2   138863  The Big Broadcast of 1936 (1935)  (no genres listed)
3   141305       Round Trip to Heaven (1992)  (no genres listed)
4   141472       The 50 Year Argument (2014)  (no genres listed)


In [6]:
# Charger la table ratings
query_ratings = """
SELECT *
FROM `master-ai-cloud.MoviePlatform.ratings`
LIMIT 50000
"""
#jai limiter a 50,000 notes pour économiser les coûts BigQuery et avoir un traitement plus rapide.
df_ratings = client.query(query_ratings).to_dataframe()
print(f"✅ Table ratings chargée : {len(df_ratings)} notes")
print(df_ratings.head())

✅ Table ratings chargée : 50000 notes
   userId  movieId  rating   timestamp
0       1      204     0.5  1217895786
1       1      256     0.5  1217895764
2       1      277     0.5  1217895772
3       1      719     0.5  1217895799
4       1    45950     0.5  1217897813


Analyses Visuelles et Statistiques

In [7]:
# Statistiques descriptives
print("=" * 50)
print("📊 STATISTIQUES GÉNÉRALES")
print("=" * 50)
print(f"Nombre total de films : {len(df_movies)}")
print(f"Nombre total de notes : {len(df_ratings)}")
print(f"Nombre d'utilisateurs uniques : {df_ratings['userId'].nunique()}")
print(f"Nombre de films notés : {df_ratings['movieId'].nunique()}")
print(f"\nNote moyenne globale : {df_ratings['rating'].mean():.2f}")
print(f"Note médiane : {df_ratings['rating'].median():.2f}")
print(f"Écart-type : {df_ratings['rating'].std():.2f}")

📊 STATISTIQUES GÉNÉRALES
Nombre total de films : 10329
Nombre total de notes : 50000
Nombre d'utilisateurs uniques : 665
Nombre de films notés : 8713

Note moyenne globale : 2.64
Note médiane : 3.00
Écart-type : 0.76


In [13]:
# Distribution des notes
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
df_ratings['rating'].value_counts().sort_index().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Distribution des Notes', fontsize=16, fontweight='bold')
plt.xlabel('Note', fontsize=12)
plt.ylabel('Nombre de notes', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('01_distribution_notes.png', dpi=150, bbox_inches='tight')
print("✅ Graphique sauvegardé : 01_distribution_notes.png")
plt.close()

✅ Graphique sauvegardé : 01_distribution_notes.png


In [14]:
# Top 20 films les plus notés
df_merged = df_ratings.merge(df_movies, on='movieId', how='left')
top_movies = df_merged['title'].value_counts().head(20)

plt.figure(figsize=(12, 8))
top_movies.plot(kind='barh', color='coral', edgecolor='black')
plt.title('Top 20 des Films les Plus Notés', fontsize=16, fontweight='bold')
plt.xlabel('Nombre de notes', fontsize=12)
plt.ylabel('Film', fontsize=12)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('02_top_films.png', dpi=150, bbox_inches='tight')
print("✅ Graphique sauvegardé : 02_top_films.png")
plt.close()

✅ Graphique sauvegardé : 02_top_films.png


In [15]:
# Activité des utilisateurs
user_activity = df_ratings['userId'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Subplot 1 : Top 20 utilisateurs
user_activity.head(20).plot(kind='bar', color='lightgreen', edgecolor='black', ax=axes[0])
axes[0].set_title('Top 20 Utilisateurs les Plus Actifs', fontsize=14, fontweight='bold')
axes[0].set_xlabel('User ID', fontsize=12)
axes[0].set_ylabel('Nombre de notes', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)

# Subplot 2 : Distribution
axes[1].hist(user_activity, bins=50, color='purple', edgecolor='black', alpha=0.7)
axes[1].set_title('Distribution du Nombre de Notes par Utilisateur', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Nombre de notes par utilisateur', fontsize=12)
axes[1].set_ylabel('Nombre d\'utilisateurs', fontsize=12)
axes[1].set_yscale('log')

plt.tight_layout()
plt.savefig('03_activite_utilisateurs.png', dpi=150, bbox_inches='tight')
print("✅ Graphique sauvegardé : 03_activite_utilisateurs.png")
print(f"Utilisateur le plus actif : {user_activity.idxmax()} avec {user_activity.max()} notes")
plt.close()

✅ Graphique sauvegardé : 03_activite_utilisateurs.png
Utilisateur le plus actif : 668 avec 3976 notes


In [16]:
# Genres les plus représentés
genres_list = []
for genres in df_movies['genres'].dropna():
    if genres != "(no genres listed)":
        genres_list.extend(genres.split('|'))

genres_counts = pd.Series(genres_list).value_counts().head(15)

plt.figure(figsize=(12, 6))
genres_counts.plot(kind='barh', color='gold', edgecolor='black')
plt.title('Top 15 des Genres les Plus Représentés', fontsize=16, fontweight='bold')
plt.xlabel('Nombre de films', fontsize=12)
plt.ylabel('Genre', fontsize=12)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('04_genres.png', dpi=150, bbox_inches='tight')
print("✅ Graphique sauvegardé : 04_genres.png")
plt.close()

✅ Graphique sauvegardé : 04_genres.png


In [17]:
# Matrice popularité vs note moyenne
movie_stats = df_merged.groupby('title').agg({
    'rating': ['count', 'mean']
}).reset_index()

movie_stats.columns = ['title', 'count', 'avg_rating']
movie_stats = movie_stats[movie_stats['count'] >= 20]

plt.figure(figsize=(12, 8))
scatter = plt.scatter(movie_stats['count'], movie_stats['avg_rating'], 
                     alpha=0.5, s=50, c=movie_stats['avg_rating'], 
                     cmap='viridis', edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='Note moyenne')
plt.title('Popularité vs Note Moyenne (films avec ≥20 notes)', fontsize=16, fontweight='bold')
plt.xlabel('Nombre de notes (popularité)', fontsize=12)
plt.ylabel('Note moyenne', fontsize=12)
plt.axhline(y=df_ratings['rating'].mean(), color='red', linestyle='--', 
            linewidth=2, label=f'Moyenne globale ({df_ratings["rating"].mean():.2f})')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('05_popularite_vs_note.png', dpi=150, bbox_inches='tight')
print("✅ Graphique sauvegardé : 05_popularite_vs_note.png")
plt.close()

✅ Graphique sauvegardé : 05_popularite_vs_note.png
